Exercise 6 of [Introduction to Robotics Mechanics and Control](http://www.mech.sharif.ir/c/document_library/get_file?uuid=5a4bb247-1430-4e46-942c-d692dead831f&groupId=14040), see page 197

[derivation of link manipulator dynamics](http://chambrierg.com/robotics/dynamics)

In [25]:
import numpy as np

class Dynamics:

    def __init__(self):
        self.l = [0.5, 0.5]
        self.m = [4.6, 2.3, 1.0]
        self.Izz = 0.1
        self.M = np.zeros((3, 3))
        self.V = np.zeros(3)
        self.G = np.zeros(3)
        
    def update(self, theta, dtheta):
        m1 = self.m[0]
        m2 = self.m[1]
        m3 = self.m[2]
        l1 = self.l[0]
        l2 = self.l[1]
        l1_s = l1**2
        l2_s = l2**2
        c1 = np.cos(theta[0])
        c2 = np.cos(theta[1])
        c12 = np.cos(theta[0] + theta[1])
        s2 = np.sin(theta[1])
        dtheta1 = dtheta[0]
        dtheta2 = dtheta[1]
        dtheta1_s = dtheta[0]**2
        dtheta2_s = dtheta[1]**2
        g = 9.8
        
        self.M.fill(self.Izz)        
        self.M[0][0] = m3*l1_s + l2_s*m2 + 2.0*l1*l2*m2*c2 + l1_s*(m1+m2)
        self.M[0][1] = l2_s*m2 + l1*l2*m2*c2 + l2_s*m2 + l1*l2*m2*c2
        self.M[1][0] = l2_s*m2 + l1*l2*c2
        
        self.V[0] = -m2*l1*l2*s2*dtheta2_s - 2.0*m2*l1*l2*s2*dtheta1*dtheta2
        self.V[1] = m2*l1*l2*s2*dtheta1_s
        
        self.G[0] = m3*l1*g*c1 + m2*l2*g*c12 + (m1+m2)*l1*g*c1
        self.G[1] = m2*l2*g*c12
    
    def get_ddtheta(self, tau, theta, dtheta):
        self.update(theta, dtheta)
        
        return np.dot(np.linalg.inv(self.M), tau - self.V - self.G)
        

Euler integrator to simulate the dynamics

In [26]:
def simulate(dynamics, theta, dtheta, tau, dt, time):
    steps = int(time/dt)
    theta = np.copy(theta)
    dtheta = np.copy(dtheta)
    
    r_theta = np.zeros(shape=(steps, len(theta)))
    r_dtheta = np.zeros(shape=(steps, len(theta)))
    r_ddtheta = np.zeros(shape=(steps, len(theta)))
    ddtheta = np.zeros(len(theta))
    
    record_step = int(0.1/dt)
    
    for t in range(steps):
        if t % record_step == 0:
            r_theta[t, :] = theta
            r_dtheta[t, :] = dtheta
            r_ddtheta[t, :] = ddtheta

        ddtheta = dynamics.get_ddtheta(tau, theta, dtheta)
        theta = theta + dtheta*dt + 0.5*ddtheta*dt*dt
        dtheta = dtheta + ddtheta*dt
        
    return r_theta, r_dtheta, r_ddtheta

**(a)** Set the initial position of the manipulator to [-90, 0, 0]. Simulate for a few seconds. Is the motion of the manipulator what you would expect ?

In [27]:
dynamics = Dynamics()
theta = np.array([-np.pi/2., 0, 0])
dtheta = np.zeros(3)
tau_c = np.zeros(3) # torque command is allways zero in this exercise.
dt = 0.001

r_theta, r_dtheta, r_ddtheta = simulate(dynamics, theta, dtheta, tau_c, dt, 5.0)

